## Use OpenCV to get frames from url

In [ ]:
import cv2

# HLS playlist URL
hls_url = "https://streamer4.brownrice.com/camdensnowbowl1/camdensnowbowl1.stream/main_playlist.m3u8"

cap = cv2.VideoCapture(hls_url)

frame_count = 0
frame_skip = 1  # Skip every 5 frames

if not cap.isOpened():
    print("Error: Could not open the HLS stream.")
else:
    while True:
        ret, frame = cap.read()
        #frame = frame[600:1050, 2400:3200] # zoom in on the chairlift
        if not ret:
            print("Frame not received, ending stream")
            break
        # frame_count += 1
        # if frame_count % frame_skip != 0:
        #     continue
        # Process your frame (e.g., people counting) here

        # For debugging, display the frame
        cv2.imshow("HLS Stream", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


## Count People in Web Cam Using YOLOv8

In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np
import torch
import os

# Import the necessary classes - this is a new issue with pytorch 2.6, could use 2.4 and not have to import all of these layers and add them to the globals list
from ultralytics.nn.tasks import DetectionModel  # Already imported for YOLOv8 models
from torch.nn.modules.container import Sequential    # For Sequential layers
from ultralytics.nn.modules.conv import Conv         # For Conv layers defined by Ultralytics
from torch.nn.modules.conv import Conv2d              # For PyTorch's Conv2d layer
from torch.nn.modules.batchnorm import BatchNorm2d
from torch.nn.modules.activation import SiLU               # PyTorch's SiLU activation
from ultralytics.nn.modules.block import C2f                       # Ultralytics' C2f block
from torch.nn.modules.container import ModuleList
from ultralytics.nn.modules.block import Bottleneck
from ultralytics.nn.modules.block import SPPF
from torch.nn.modules.pooling import MaxPool2d
from torch.nn.modules.upsampling import Upsample
from ultralytics.nn.modules.conv import Concat
from ultralytics.nn.modules.head import Detect
from ultralytics.nn.modules.block import DFL
torch.serialization.add_safe_globals([
    DetectionModel, Sequential, Conv, Conv2d, BatchNorm2d, SiLU, C2f, ModuleList,
    Bottleneck, SPPF, MaxPool2d, Upsample, Concat, Detect, DFL
])

from src.config.config import frame_capture_settings


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load the pretrained YOLOv8 model and move it to the appropriate device
model = YOLO(os.path.join(frame_capture_settings.model_path, frame_capture_settings.model_name)).to(device)
print(model.names)

# Define the HLS stream URL (the direct stream URL you extracted)
hls_url = "https://streamer4.brownrice.com/camdensnowbowl1/camdensnowbowl1.stream/main_playlist.m3u8"

# Open the video stream
cap = cv2.VideoCapture(hls_url)
if not cap.isOpened():
    print("Error: Could not open the video stream.")
    exit()

frame_skip = 20  # Process every 20th frame
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
    frame_count += 1

    # Skip frames until we hit the desired interval
    if frame_count % frame_skip != 0:
        continue

    # Run YOLOv8 inference on the current frame.
    results = model(frame)
    
    # We'll use a copy of the frame to draw annotations.
    annotated_frame = frame.copy()
    people_count = 0

    # Process each detection result
    for result in results:
        if result.boxes is not None:
            boxes = result.boxes.data.cpu().numpy()  # shape: (num_boxes, 6)
            for box in boxes:
                x1, y1, x2, y2, conf, cls = box
                # In COCO, the 'person' class typically has an id of 0.
                if int(cls) >= 0:
                    bbox_color = (0, 255, 0)  # Green
                    if cls != 0:
                        bbox_color = (255, 0, 0)
                    cv2.rectangle(annotated_frame, (int(x1), int(y1)), (int(x2), int(y2)), bbox_color, 2)
                    people_count += 1
                    # Draw the bounding box and label on the frame.
                    cv2.putText(
                        annotated_frame,
                        f"{int(cls)} {conf:.2f}",
                        (int(x1), int(y1) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (0, 255, 0),
                        2,
                    )
    
    # Overlay the people count on the frame.
    cv2.putText(
        annotated_frame,
        f"People Count: {people_count}",
        (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 0, 255),
        2,
    )
    
    # Display the annotated frame
    cv2.imshow("YOLOv8 People Counting", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the stream and close windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np
import torch
import os

from src.config.config import frame_capture_settings


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load the pretrained YOLOv8 model and move it to the appropriate device
model1 = YOLO('finetune/yolo11l_webcam_finetune/weights/best.pt').to(device)
model2 = YOLO('src/models/yolo11l.pt').to(device)
print(model.names)

# Define the HLS stream URL (the direct stream URL you extracted)
hls_url = "https://streamer4.brownrice.com/camdensnowbowl1/camdensnowbowl1.stream/main_playlist.m3u8"

# Open the video stream
cap = cv2.VideoCapture(hls_url)
if not cap.isOpened():
    print("Error: Could not open the video stream.")
    exit()

frame_skip = 2  # Process every 20th frame
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
    frame_count += 1

    # Skip frames until we hit the desired interval
    if frame_count % frame_skip != 0:
        continue

    # Run YOLOv8 inference on the current frame.
    results = model(frame)
    
    # We'll use a copy of the frame to draw annotations.
    annotated_frame = frame.copy()
    people_count = 0

    # Process each detection result
    for result in results:
        if result.boxes is not None:
            boxes = result.boxes.data.cpu().numpy()  # shape: (num_boxes, 6)
            for box in boxes:
                x1, y1, x2, y2, conf, cls = box
                # In COCO, the 'person' class typically has an id of 0.
                if conf < 0.42:
                    continue
                if int(cls) >= 0:
                    bbox_color = (0, 255, 0)  # Green
                    if cls != 0:
                        bbox_color = (255, 0, 0)
                    cv2.rectangle(annotated_frame, (int(x1), int(y1)), (int(x2), int(y2)), bbox_color, 2)
                    people_count += 1
                    # Draw the bounding box and label on the frame.
                    cv2.putText(
                        annotated_frame,
                        f"{int(cls)} {conf:.2f}",
                        (int(x1), int(y1) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (0, 255, 0),
                        2,
                    )
    
    # Overlay the people count on the frame.
    cv2.putText(
        annotated_frame,
        f"People Count: {people_count}",
        (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 0, 255),
        2,
    )
    
    # Display the annotated frame
    cv2.imshow("YOLOv8 People Counting", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the stream and close windows
cap.release()
cv2.destroyAllWindows()


## Create a dataset we can use to fine tune the model for counting people on this web cam
Saves every 1000th frame into the dataset/images/train folder and the associated YOLOv8 detected people annotations into the dataset/labels/train folder

Idea is we then go through these images after we gather a lot and improve upon the annotations. Then we fine tune the YOLO model with this data.

In [ ]:
import os
import cv2
import time
from ultralytics import YOLO
import torch

# Set up safe globals for PyTorch 2.6+ (include only if necessary)
from ultralytics.nn.tasks import DetectionModel
from torch.nn.modules.container import Sequential, ModuleList
from ultralytics.nn.modules.conv import Conv, Concat
from torch.nn.modules.conv import Conv2d
from torch.nn.modules.batchnorm import BatchNorm2d
from torch.nn.modules.activation import SiLU
from ultralytics.nn.modules.block import C2f, Bottleneck, SPPF, DFL
from torch.nn.modules.pooling import MaxPool2d
from torch.nn.modules.upsampling import Upsample
from ultralytics.nn.modules.head import Detect
torch.serialization.add_safe_globals([
    DetectionModel, Sequential, Conv, Conv2d, BatchNorm2d, SiLU, C2f, ModuleList,
    Bottleneck, SPPF, MaxPool2d, Upsample, Concat, Detect, DFL
])

# Adjustable parameters
WEBCAM_URL = "https://streamer4.brownrice.com/camdensnowbowl1/camdensnowbowl1.stream/main_playlist.m3u8"
FRAME_INTERVAL = 1000  # Process every 1000th frame
CONF_THRESHOLD = 0.4   # Confidence threshold
IMG_DIR = 'dataset/images/train'
LABEL_DIR = 'dataset/labels/train'


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load YOLOv8 model
model = YOLO("yolov8n.pt").to(device)

cap = cv2.VideoCapture(WEBCAM_URL)
if not cap.isOpened():
    print("Error: Could not open the video stream.")
    exit()

frame_count = 0

while True:
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    frame_count += 1
    # Process only every FRAME_INTERVAL frame
    if frame_count % FRAME_INTERVAL != 1:
        continue

    # Save the raw frame
    timestamp = int(time.time())
    img_filename = os.path.join(IMG_DIR, f"frame_{frame_count}_{timestamp}.jpg")
    cv2.imwrite(img_filename, frame)
    print(f"Saved image: {img_filename}")

    # Run YOLOv8 inference on the frame
    results = model(frame)
    height, width = frame.shape[:2]

    # Open a .txt file for writing the annotations in YOLO format
    txt_filename = os.path.splitext(img_filename)[0] + ".txt"
    txt_filename = txt_filename.replace("images", "labels")
    with open(txt_filename, "w") as f:
        for result in results:
            if result.boxes is not None:
                boxes = result.boxes.data.cpu().numpy()  # each row: [x1, y1, x2, y2, conf, cls]
                for box in boxes:
                    x1, y1, x2, y2, conf, cls = box
                    if conf >= CONF_THRESHOLD and int(cls) == 0:  # Only person (class 0)
                        # Convert bounding box to YOLO format (normalized)
                        x_center = ((x1 + x2) / 2.0) / width
                        y_center = ((y1 + y2) / 2.0) / height
                        bbox_width = (x2 - x1) / width
                        bbox_height = (y2 - y1) / height
                        # Write annotation line: class x_center y_center width height
                        f.write(f"0 {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}\n")
    print(f"Saved annotations: {txt_filename}")

    # Optionally, display the frame (with no annotations drawn)
    #cv2.imshow("Dataset Collection", frame)


cap.release()
cv2.destroyAllWindows()

In [ ]:
import os
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt

# Hardcoded folder path (change this to your folder)
folder_path = 'datasets/dataset_webcam/labels/train'
model_path = 'finetune/yolo11l_external_finetune/weights/best.pt'
model = YOLO(model_path)

def get_yolo_boxes_from_annotation_file(annotation_file):
    boxes = []
    if not os.path.exists(annotation_file):
        print(f"Annotation file not found: {annotation_file}")
        return boxes  # return empty list if no file
    with open(annotation_file, 'r') as f:
        lines = f.readlines()
    for line in lines:
        # Each line: "class_id x_center y_center width height"
        box = line.strip().split(' ')
        if len(box) != 5:
            continue  # skip invalid lines
        box = [float(x) for x in box]
        boxes.append(box)
    return boxes

def draw_box(yolo_box, height, width, ax):
        """Draw a rectangle for a box in YOLO format and return the patch."""
        xc = yolo_box[1] * width
        yc = yolo_box[2] * height
        w = yolo_box[3] * width
        h = yolo_box[4] * height
        x0 = xc - w/2
        y0 = yc - h/2
        patch = plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2)
        ax.add_patch(patch)
        

# Loop over all items in the folder
for filename in os.listdir(folder_path):
    annotation_path = os.path.join(folder_path, filename)
    img_filename = filename.replace('.txt', '.jpg')
    img = cv2.imread(os.path.join(folder_path, img_filename))
    if img is None:
        print(f"Image {img_filename} not found.")
        continue
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    boxes = get_yolo_boxes_from_annotation_file(annotation_path)
    if not boxes:
        print(f"No boxes found in {annotation_path}")
        continue

    fig, ax = plt.subplots()
    ax.imshow(img)
    height, width, _ = img.shape

    for box in boxes:
        draw_box(box, height, width, ax)
    
    break


In [5]:
from ultralytics import YOLO
import torch

model = YOLO('finetune/yolo11l_webcam_finetune/weights/best.pt')
total_params = sum(p.numel() for p in model.parameters())
print("Total parameters:", total_params)

model_size_bytes = total_params * 4  # assuming float32
model_size_mb = model_size_bytes / (1024**2)
print("Approximate model size (MB):", model_size_mb)


Total parameters: 25311251
Approximate model size (MB): 96.55475997924805


In [6]:
param = next(model.parameters())
print(param.dtype, param.element_size())


torch.float32 4


In [9]:
model.info()

YOLO11l summary: 631 layers, 25,311,251 parameters, 0 gradients, 87.3 GFLOPs


(631, 25311251, 0, 87.27372799999999)

In [8]:
x = [p.numel() for p in model.parameters()]
print(len(x))

514


In [11]:
actual_model_size_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
actual_model_size_mb = actual_model_size_bytes / (1024**2)
print("Actual in-memory model size (MB):", actual_model_size_mb)


Actual in-memory model size (MB): 96.55475997924805


In [ ]:
from memory_profiler import memory_usage
def run_inference():
    # Run a single inference, e.g.:
    output = model(input_tensor)
    return output

mem_usage = memory_usage(proc=run_inference, interval=0.1)
print("Memory usage (MB):", max(mem_usage))


In [5]:
import time
timestamp = 1740237112.471
datetime_str = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timestamp))
print(datetime_str)

2025-02-22 10:11:52


In [17]:
import os
for file in sorted(os.listdir("/home/ddd/people-counter/demo/raw_frames")):
    print(file, end="\t - \t")
    timestr = float(file[:-4])
    #print(timestr)
    print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timestr)))
    # print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(file.split(".")[:1])))

1740236585.140.jpg	 - 	2025-02-22 10:03:05
1740236589.066.jpg	 - 	2025-02-22 10:03:09
1740236592.881.jpg	 - 	2025-02-22 10:03:12
1740236596.312.jpg	 - 	2025-02-22 10:03:16
1740236600.234.jpg	 - 	2025-02-22 10:03:20
1740236603.762.jpg	 - 	2025-02-22 10:03:23
1740236607.111.jpg	 - 	2025-02-22 10:03:27
1740236612.281.jpg	 - 	2025-02-22 10:03:32
1740236621.283.jpg	 - 	2025-02-22 10:03:41
1740236624.909.jpg	 - 	2025-02-22 10:03:44
1740236626.989.jpg	 - 	2025-02-22 10:03:46
1740236628.670.jpg	 - 	2025-02-22 10:03:48
1740236632.240.jpg	 - 	2025-02-22 10:03:52
1740236635.815.jpg	 - 	2025-02-22 10:03:55
1740236639.568.jpg	 - 	2025-02-22 10:03:59
1740236643.253.jpg	 - 	2025-02-22 10:04:03
1740236646.711.jpg	 - 	2025-02-22 10:04:06
1740236654.017.jpg	 - 	2025-02-22 10:04:14
1740236658.052.jpg	 - 	2025-02-22 10:04:18
1740236661.702.jpg	 - 	2025-02-22 10:04:21
1740236665.286.jpg	 - 	2025-02-22 10:04:25
1740236669.513.jpg	 - 	2025-02-22 10:04:29
1740236673.810.jpg	 - 	2025-02-22 10:04:33
1740236678.